# Basic experiment over walk, run and bike ride

In [1]:
import sys
import warnings
import math as m
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import acf

import scipy
from scipy.signal import find_peaks
from scipy.spatial import distance_matrix

sys.path.insert(0, "./../")
from qppe.visuals import (
    plot_short_timeseries,
    plot_phase_trajectory,
    plot_phase_trajectory_and_phase
)
from qppe.qppe import (
    delay_embedding_matrix,
    QPPE
)

In [2]:
warnings.simplefilter('ignore')

In [3]:
def prepare_time_series(path, centred = True):
    data = pd.read_csv(path, delimiter =';', decimal=',')
    
    frequency = len(data)/(data['time'].values[-1]-data['time'].values[0])
    time_series = ((data['X_value'].values)**2 + (data['Y_value'].values)**2 + (data['Z_value'].values)**2)**.5
    
    if centred:
        time_series = (time_series - np.mean(time_series))
        
    time_points = (data['time'].values).astype(float).reshape([-1,])
    time_points = np.linspace(0,time_points[-1]-time_points[0],len(time_series))

    return time_series,time_points

In [ ]:
x_acc, t = prepare_time_series('./../data/basic_experiment_walk.csv', centred = False)
x_acc = x_acc[7000:20000]

X_h = delay_embedding_matrix(x_acc, 500)

pca = PCA(n_components = 4)
X_PCA = pca.fit_transform(X_h)

phase_trajectory = X_PCA[::3,:3]

result_dict = QPPE(x = x_acc, update_coef = 3, return_expectation_model = True)

 60%|██████    | 7518/12501 [00:39<00:07, 706.60it/s] 

In [ ]:
phase_trajectory = X_PCA[::5,:3]
expextation_values = result_dict['expectation'][:500:,:3]
phase_history = result_dict['phase'][::5]

In [ ]:
plt.rcParams['figure.figsize'] = 5, 5
plot_short_timeseries(x_acc[:1000], 500)
plot_phase_trajectory(phase_trajectory)

plt.rcParams['figure.figsize'] = 10, 10
plot_phase_trajectory_and_phase(
    phase_trajectory,
    expextation_values,
    phase_history,
    rotation = (0, 30, 0)
)